In [33]:
# Jupyter-ready: Load 8 HDF5 B-field files, build complex fields, crop, and plot one (magnitude & phase)
import h5py
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# -----------------------------
# User-configurable parameters
# -----------------------------
# Filenames follow: "B1+ (f=127.7) [i] 500 W stim.h5" for i = 1..8 (adjust if your names differ)
base_pattern = "h-field (f=127.7) [{i}].h5"

# Which component to use for the complex field (based on your header, z carries the complex values)
component = "z"  # choose from: "x", "y", "z"

# Cropping: centered square crop
crop_size = 250  # adjust as needed


# Color scaling (None -> autoscale)
mag_vmin, mag_vmax = None, None
phase_vmin, phase_vmax = -180, 180  # degrees; set to None for autoscale

# -----------------------------
# Load all fields into b_raw
# -----------------------------
b_raw = []   # list of 8 complex 2D arrays
pos_raw = [] # list of 8 (optional) position arrays if needed later

def load_complex_field_from_h5(path, comp="z"):
    with h5py.File(path, "r") as f:
        bf = f["H-Field"]  # structured: ('x':('re','im'), 'y':(...), 'z':(...))
        # Infer grid size (assumes square grid)
        n = bf.shape[0]
        s = int(round(np.sqrt(n)))
        if s * s != n:
            raise ValueError(f"Dataset size {n} is not a perfect square; cannot reshape to 2D grid.")
        # Build complex array from requested component
        re = bf[comp]["re"][:]
        im = bf[comp]["im"][:]
        arr = (re + 1j * im).reshape(s, s)

        # Optionally parse positions (not required for plotting, but kept if needed)
        if "Position" in f:
            pos = f["Position"][:]  # dtype [('x','<f8'),('y','<f8'),('z','<f8')]
            pos = {k: pos[k].reshape(s, s) for k in ("x", "y", "z")}
        else:
            pos = None

    return arr, pos

# Gather 8 files
file_paths = [Path(base_pattern.format(i=i)) for i in range(1, 17)]

for p in file_paths:
    arr, pos = load_complex_field_from_h5(p, comp=component)
    b_raw.append(arr)
    pos_raw.append(pos)

# -----------------------------
# Centered square cropping
# -----------------------------
def center_crop_square(img, size):
    h, w = img.shape
    s = min(size, h, w)  # safe if requested crop is larger than image
    y0 = (h - s) // 2
    x0 = (w - s) // 2
    return img[y0:y0+s, x0:x0+s]

b_crop = [center_crop_square(arr, crop_size)* 1.257E-6 for arr in b_raw]   # list of 8 cropped complex fields





In [34]:
# -----------------------------
# Plot one field (index [1]) - magnitude & phase
# -----------------------------
# Plot settings for field [1]
plot_field_index = 1  # 1..8 (user-facing); internally we convert to 0-based
idx = plot_field_index - 1  # convert 1..8 to 0..7
field_c = b_crop[idx]

mag = np.abs(field_c)
phase_deg = np.angle(field_c, deg=True)
# Magnitude plot
plt.figure()
im0 = plt.imshow(mag, cmap="jet", origin="lower", vmin=mag_vmin, vmax=mag_vmax)
plt.title(f"Field [{plot_field_index}] Magnitude (component={component})")
plt.colorbar(im0, label="|B| (arb. units)")
plt.xlabel("x (px)")
plt.ylabel("y (px)")
plt.tight_layout()

# # Phase plot
# plt.figure()
# im1 = plt.imshow(phase_deg, cmap="jet", origin="lower", vmin=phase_vmin, vmax=phase_vmax)
# plt.title(f"Field [{plot_field_index}] Phase (degrees, component={component})")
# plt.colorbar(im1, label="Phase (deg)")
# plt.xlabel("x (px)")
# plt.ylabel("y (px)")
# plt.tight_layout()

In [47]:
# JupyterLab-Robust Interactive with:
# - Default mode = "Sum of Squares"
# - Phase sliders default [0,45,90,130,180,225,270,315]
# - Three plots: Magnitude, Original Phase, Wrapped Phase
# - Extra dropdown to toggle Magnitude colorbar scale: "Linear" or "Log"

import numpy as np
import matplotlib
matplotlib.use("Agg")  # offscreen, robust in JupyterLab
import matplotlib.pyplot as plt
from io import BytesIO
from ipywidgets import VBox, HBox, FloatSlider, IntSlider, Dropdown, Button, Layout, Label, Image
from IPython.display import display

# ---------------------------------------------------------
# Expect B_MAG and B_PHASE_ORIG
# ---------------------------------------------------------
try:
    B_MAG
    B_PHASE_ORIG
except NameError:
    try:
        b_crop
    except NameError:
        raise RuntimeError("b_crop not found. Run the loader/crop cell first.")
    b_mag = [np.abs(c) for c in b_crop]
    b_phase_orig = [np.angle(c, deg=True) for c in b_crop]
    B_MAG = np.stack(b_mag, axis=0)
    B_PHASE_ORIG = np.stack(b_phase_orig, axis=0)

H, W = B_MAG.shape[1], B_MAG.shape[2]

# ---------------------------------------------------------
# Controls
# ---------------------------------------------------------
mode_dd = Dropdown(
    options=["Sum of Squares", "Sum"],
    value="Sum of Squares",
    description="Mode:",
    layout=Layout(width="220px")
)

scale_dd = Dropdown(
    options=["Linear", "Log"],
    value="Linear",
    description="Scale:",
    layout=Layout(width="220px")
)

# Default phase values
# default_phases = np.linspace(0, 360, 16, endpoint=False)
# 1. Create the 8 values for the odd-numbered positions
# (np.linspace(0, 360, 8, endpoint=False) gives [0, 45, 90, ... 315])
odd_values = np.linspace(0, 360, 8, endpoint=False)
# 2. Calculate the 8 values for the even-numbered positions
# This is the prior odd value + 180, wrapped around 360
# The modulo operator (%) handles the wrap-around perfectly.
even_values = (odd_values + 180) % 360
# 3. Create an empty 16-element array to hold the result
result = np.empty(16)
# 4. Assign the values using slicing
# [0::2] means "start at index 0, go to the end, in steps of 2"
result[0::2] = odd_values
# [1::2] means "start at index 1, go to the end, in steps of 2"
result[1::2] = even_values
default_phases = result



mag_sliders = [
    FloatSlider(value=1.0, min=0.0, max=3.0, step=0.01,
                description=f"Mag {i+1}", continuous_update=False,
                layout=Layout(width="300px"))
    for i in range(16)
]
phs_sliders = [
    IntSlider(value=default_phases[i], min=0, max=360, step=1,
              description=f"Phase {i+1}", continuous_update=False,
              layout=Layout(width="300px"))
    for i in range(16)
]
reset_btn = Button(description="Reset weights & phases", layout=Layout(width="250px"))

def reset_sliders(_=None):
    for s in mag_sliders: s.value = 1.0
    for i, s in enumerate(phs_sliders): s.value = default_phases[i]
reset_btn.on_click(reset_sliders)

# ---------------------------------------------------------
# Math
# ---------------------------------------------------------
def combine_fields(mode, mags, phs_deg):
    mags = np.asarray(mags, dtype=np.float64)
    phs_rad = np.deg2rad(np.asarray(phs_deg, dtype=np.float64))
    phi0_rad = np.deg2rad(B_PHASE_ORIG)

    mag_w = mags[:, None, None] * B_MAG
    phase_total = phi0_rad + phs_rad[:, None, None]
    complex_terms = mag_w * np.exp(1j * phase_total)
    complex_sum = np.sum(complex_terms, axis=0)

    phase_comb_deg = np.angle(complex_sum, deg=True)
    if mode == "Sum":
        mag_comb = np.abs(complex_sum)
    else:  # Sum of Squares
        mag_comb = np.sum(mag_w**2, axis=0)
    return mag_comb, phase_comb_deg

# ---------------------------------------------------------
# Render helpers
# ---------------------------------------------------------
def render_image(arr, title="", cmap="jet", vmin=None, vmax=None, cbar_label="", phase=False, wrap=False):
    fig, ax = plt.subplots(figsize=(4.2, 4), dpi=120 , constrained_layout=False)
    if phase and not wrap and (vmin is None or vmax is None):
        vmin, vmax = -180, 180
    if wrap and (vmin is None or vmax is None):
        vmin, vmax = 0, 360
    im = ax.imshow(arr, cmap=cmap, origin="lower", vmin=vmin, vmax=vmax)
    cb = plt.colorbar(im, ax=ax)
    if cbar_label: cb.set_label(cbar_label)
    ax.set_title(title)
    ax.set_xlabel("x (px)")
    ax.set_ylabel("y (px)")
    fig.tight_layout()

    buf = BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    plt.close(fig)
    return buf.getvalue()

# ---------------------------------------------------------
# Outputs
# ---------------------------------------------------------
img_mag        = Image(format="png")
img_ph         = Image(format="png")
img_ph_wrapped = Image(format="png")

def update(_=None):
    mode = mode_dd.value
    scale_mode = scale_dd.value
    mags = [s.value for s in mag_sliders]
    phs  = [s.value for s in phs_sliders]

    mag_comb, phase_comb_deg = combine_fields(mode, mags, phs)
    phase_wrapped = np.mod(phase_comb_deg, 360.0)

    # Handle magnitude scale
    if scale_mode == "Log":
        mag_plot = np.log10(np.clip(mag_comb, 1e-12, None))
        vmin = float(np.nanmin(mag_plot))
        vmax = float(np.nanmax(mag_plot))
        vmin = -5
        vmax = 0
        title_mag = f"Combined Magnitude (Log10) — Mode: {mode}"
        cbar_lab = "log10(|Magnitude|)"
    else:
        mag_plot = mag_comb
        vmin = float(np.nanmin(mag_plot))
        vmax = float(np.nanmax(mag_plot))
        title_mag = f"Combined Magnitude — Mode: {mode}"
        cbar_lab = "Magnitude (arb.)"

    img_mag.value = render_image(mag_plot, title=title_mag,
                                 cmap="jet", vmin=vmin, vmax=vmax, cbar_label=cbar_lab)
    img_ph.value  = render_image(phase_comb_deg, title="Combined Phase (original)",
                                 cmap="jet", phase=True, cbar_label="Phase (deg)")
    img_ph_wrapped.value = render_image(phase_wrapped, title="Combined Phase (wrapped 0–360°)",
                                        cmap="jet", wrap=True, cbar_label="Phase (deg)")

# Wire events
controls = [mode_dd, scale_dd, reset_btn] + mag_sliders + phs_sliders
for w in controls:
    if hasattr(w, "observe"):
        w.observe(update, names="value")

# Initial compute
update()

ctrl_left  = VBox([Label("Mode & Scale"), HBox([mode_dd, scale_dd, reset_btn]), Label("Magnitude Weights (0–3)")] + mag_sliders)
ctrl_right = VBox([Label("Phase Shifts (deg, 0–360)")] + phs_sliders)
plots_box  = HBox([img_mag, img_ph, img_ph_wrapped])

display(HBox([ctrl_left, ctrl_right], layout=Layout(justify_content="space-between", align_items="flex-start")))
display(plots_box)

In [36]:
# ==== Block: 3×8 grid of ALL 8 FIELDS (top=magnitude, middle=phase, bottom=wrapped phase) ====
# Robust in JupyterLab (Agg backend). Adds a "Scale" dropdown for magnitude (Linear/Log).
# Expects B_MAG and B_PHASE_ORIG to already exist (from earlier cells).
# If they don't, it will try to build them from b_crop (list of 8 complex 2D arrays).

import numpy as np
import matplotlib
matplotlib.use("Agg")  # ensure non-GUI backend
import matplotlib.pyplot as plt
from io import BytesIO
from ipywidgets import Dropdown, HBox, VBox, Layout, Label, Image
from IPython.display import display

# Prepare data if needed
try:
    B_MAG
    B_PHASE_ORIG
except NameError:
    try:
        b_crop
    except NameError:
        raise RuntimeError("b_crop not found. Run the loader/crop cell first.")
    b_mag = [np.abs(c) for c in b_crop]
    b_phase_orig = [np.angle(c, deg=True) for c in b_crop]
    B_MAG = np.stack(b_mag, axis=0)               # (8,H,W)
    B_PHASE_ORIG = np.stack(b_phase_orig, axis=0) # (8,H,W)

# Wrapped phases
B_PHASE_WRAP = np.mod(B_PHASE_ORIG, 360.0)

# Widgets
scale_dd = Dropdown(options=["Linear", "Log"], value="Linear",
                    description="Magnitude Scale:", layout=Layout(width="240px"))

img_grid = Image(format="png")

def render_grid(scale="Linear"):
    """Return PNG bytes for a 3x8 grid: top row magnitude, middle phase, bottom wrapped phase."""
    # Handle magnitude scaling
    if scale == "Log":
        mag_to_plot = np.log10(np.clip(B_MAG, 1e-12, None))
        mag_label = "log10(|B|)"
        mag_vmin = float(np.nanmin(mag_to_plot))
        mag_vmax = float(np.nanmax(mag_to_plot))
    else:
        mag_to_plot = B_MAG
        mag_label = "|B|"
        mag_vmin = float(np.nanmin(mag_to_plot))
        mag_vmax = float(np.nanmax(mag_to_plot))

    phase_to_plot = B_PHASE_ORIG  # degrees
    wrapped_to_plot = B_PHASE_WRAP
    ph_vmin, ph_vmax = -180, 180
    phw_vmin, phw_vmax = 0, 360

    # Build figure
    fig, axes = plt.subplots(3, 16, figsize=(16, 6.8), dpi=120, constrained_layout=True)

    # Top row: magnitudes
    ims_mag = []
    for i in range(16):
        ax = axes[0, i]
        im = ax.imshow(mag_to_plot[i], cmap="jet", origin="lower", vmin=mag_vmin, vmax=mag_vmax)
        ax.set_title(f"Field {i+1} — Mag")
        ax.set_xticks([]); ax.set_yticks([])
        ims_mag.append(im)

    # Middle row: phases
    ims_ph = []
    for i in range(16):
        ax = axes[1, i]
        im = ax.imshow(phase_to_plot[i], cmap="jet", origin="lower", vmin=ph_vmin, vmax=ph_vmax)
        ax.set_title(f"Field {i+1} — Phase")
        ax.set_xticks([]); ax.set_yticks([])
        ims_ph.append(im)

    # Bottom row: wrapped phases
    ims_phw = []
    for i in range(16):
        ax = axes[2, i]
        im = ax.imshow(wrapped_to_plot[i], cmap="jet", origin="lower", vmin=phw_vmin, vmax=phw_vmax)
        ax.set_title(f"Field {i+1} — Wrapped")
        ax.set_xticks([]); ax.set_yticks([])
        ims_phw.append(im)

    # Shared colorbars: one for magnitudes, one for phases, one for wrapped
    cbar_ax1 = fig.add_axes([0.91, 0.71, 0.015, 0.20])
    cbar1 = fig.colorbar(ims_mag[-1], cax=cbar_ax1)
    cbar1.set_label(mag_label)

    cbar_ax2 = fig.add_axes([0.91, 0.40, 0.015, 0.20])
    cbar2 = fig.colorbar(ims_ph[-1], cax=cbar_ax2)
    cbar2.set_label("Phase (deg)")

    cbar_ax3 = fig.add_axes([0.91, 0.09, 0.015, 0.20])
    cbar3 = fig.colorbar(ims_phw[-1], cax=cbar_ax3)
    cbar3.set_label("Wrapped (deg)")

    # Save to PNG
    buf = BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    plt.close(fig)
    return buf.getvalue()

def update(_=None):
    img_grid.value = render_grid(scale=scale_dd.value)

scale_dd.observe(update, names="value")

# Initial render + layout
update()
display(VBox([HBox([scale_dd]), img_grid]))

In [38]:
# ==== Block: Combine 16 fields into 8 Segments (pairwise addition) ====
# Each Segment = Field(2n-1) + Field(2n)
# Then display 3×8 grid (top: magnitude, middle: phase, bottom: wrapped phase)
# Robust in JupyterLab (Agg backend). Adds "Linear" / "Log" scale option for magnitude.

import numpy as np
import matplotlib
matplotlib.use("Agg")  # ensure non-GUI backend
import matplotlib.pyplot as plt
from io import BytesIO
from ipywidgets import Dropdown, HBox, VBox, Layout, Image
from IPython.display import display

# ---------------------------------------------------------
# Prepare data
# ---------------------------------------------------------
try:
    B_MAG
    B_PHASE_ORIG
except NameError:
    try:
        b_crop
    except NameError:
        raise RuntimeError("b_crop not found. Run the loader/crop cell first.")
    b_mag = [np.abs(c) for c in b_crop]
    b_phase_orig = [np.angle(c, deg=True) for c in b_crop]
    B_MAG = np.stack(b_mag, axis=0)
    B_PHASE_ORIG = np.stack(b_phase_orig, axis=0)

if B_MAG.shape[0] != 16:
    raise ValueError(f"Expected 16 fields, found {B_MAG.shape[0]}.")

# ---------------------------------------------------------
# Combine fields pairwise
# ---------------------------------------------------------
B_COMPLEX = B_MAG * np.exp(1j * np.deg2rad(B_PHASE_ORIG))  # reconstruct complex fields
B_SEGMENTS = []  # will contain 8 combined fields
for i in range(0, 16, 2):
    seg = B_COMPLEX[i] + B_COMPLEX[i + 1]
    B_SEGMENTS.append(seg)
B_SEGMENTS = np.stack(B_SEGMENTS, axis=0)  # shape (8, H, W)

# Compute derived maps
B_SEG_MAG = np.abs(B_SEGMENTS)
B_SEG_PHASE = np.angle(B_SEGMENTS, deg=True)
B_SEG_WRAP = np.mod(B_SEG_PHASE, 360.0)

# ---------------------------------------------------------
# Widgets
# ---------------------------------------------------------
scale_dd = Dropdown(options=["Linear", "Log"], value="Log",
                    description="Magnitude Scale:", layout=Layout(width="240px"))
# Add after the 'scale_dd' definition
from ipywidgets import FloatSlider

# # Slider for magnitude colorbar scaling factor (multiplier on max value)
# mag_scale_slider = FloatSlider(
#     value=1.0, min=0.1, max=3.0, step=0.05,
#     description="Colorbar Scale:",
#     continuous_update=False,
#     readout_format=".2f",
#     layout=Layout(width="300px")
# )
# display(VBox([HBox([scale_dd, mag_scale_slider]), img_grid]))
img_grid = Image(format="png")

def render_grid(scale="Linear"):
    """Render PNG for 3×8 grid (top: mag, mid: phase, bottom: wrapped)."""
    if scale == "Log":
        mag_plot = np.log10(np.clip(B_SEG_MAG, 1e-12, None))
        mag_label = "log10(|B|)"
        mag_vmin, mag_vmax = float(np.nanmin(mag_plot)), float(np.nanmax(mag_plot))
        mag_vmin = -5
        mag_vmax = 0
    else:
        mag_plot = B_SEG_MAG
        mag_label = "|B|"
        mag_vmin, mag_vmax = float(np.nanmin(mag_plot)), float(np.nanmax(mag_plot))
        mag_vmin = float(np.nanmin(mag_plot))
        mag_vmax = 0.5

    ph_vmin, ph_vmax = -180, 180
    phw_vmin, phw_vmax = 0, 360

    fig, axes = plt.subplots(3, 8, figsize=(16, 6.8), dpi=120, constrained_layout=False)

    # Row 1: magnitude
    for i in range(8):
        ax = axes[0, i]
        im = ax.imshow(mag_plot[i], cmap="jet", origin="lower", vmin=mag_vmin, vmax=mag_vmax)
        ax.set_title(f"Segment {i+1} — Mag")
        ax.set_xticks([]); ax.set_yticks([])
    cbar_ax1 = fig.add_axes([0.91, 0.71, 0.015, 0.20])
    cbar1 = fig.colorbar(im, cax=cbar_ax1)
    cbar1.set_label(mag_label)

    # Row 2: phase
    for i in range(8):
        ax = axes[1, i]
        im = ax.imshow(B_SEG_PHASE[i], cmap="jet", origin="lower", vmin=ph_vmin, vmax=ph_vmax)
        ax.set_title(f"Segment {i+1} — Phase")
        ax.set_xticks([]); ax.set_yticks([])
    cbar_ax2 = fig.add_axes([0.91, 0.40, 0.015, 0.20])
    cbar2 = fig.colorbar(im, cax=cbar_ax2)
    cbar2.set_label("Phase (deg)")

    # Row 3: wrapped phase
    for i in range(8):
        ax = axes[2, i]
        im = ax.imshow(B_SEG_WRAP[i], cmap="jet", origin="lower", vmin=phw_vmin, vmax=phw_vmax)
        ax.set_title(f"Segment {i+1} — Wrapped")
        ax.set_xticks([]); ax.set_yticks([])
    cbar_ax3 = fig.add_axes([0.91, 0.09, 0.015, 0.20])
    cbar3 = fig.colorbar(im, cax=cbar_ax3)
    cbar3.set_label("Wrapped (deg)")

    buf = BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    plt.close(fig)
    return buf.getvalue()

def update(_=None):
    img_grid.value = render_grid(scale=scale_dd.value)

scale_dd.observe(update, names="value")

# Initial render + layout
update()
display(VBox([HBox([scale_dd]), img_grid]))

###### 

#

In [46]:
# === JupyterLab-Robust Interactive: 32-field system with EXPANDED WIDTH ===
# - Starts from 16 base fields (B_MAG, B_PHASE_ORIG) of shape (16, H, W)
# - Builds 16 shifted copies (shift right by `shift_pixels`) into an EXPANDED canvas of width W+shift
# - Creates 16 originals (expanded) + 16 (original shifted and added) => total 32 fields, each with its own sliders
# - Combines fields in either "Sum of Squares" or "Sum" mode
# - Shows Magnitude, Original Phase, and Wrapped Phase (0–360°)
# - Robust in JupyterLab (Agg backend): renders to PNGs inside ipywidgets.Image

import numpy as np
import matplotlib
matplotlib.use("Agg")  # offscreen, robust
import matplotlib.pyplot as plt
from io import BytesIO
from ipywidgets import VBox, HBox, FloatSlider, IntSlider, Dropdown, Button, Layout, Label, Image
from IPython.display import display

# ---------------------------------------------------------
# Expect B_MAG and B_PHASE_ORIG from earlier (shape (16, H, W))
# ---------------------------------------------------------
try:
    B_MAG
    B_PHASE_ORIG
except NameError:
    try:
        b_crop
    except NameError:
        raise RuntimeError("b_crop not found. Run the loader/crop cell first.")
    B_MAG = np.stack([np.abs(c) for c in b_crop], axis=0)
    B_PHASE_ORIG = np.stack([np.angle(c, deg=True) for c in b_crop], axis=0)

if B_MAG.shape[0] != 16:
    raise ValueError(f"Expected 16 base fields, got {B_MAG.shape[0]}.")

H, W = B_MAG.shape[1], B_MAG.shape[2]

# ---------------------------------------------------------
# CONFIG
# ---------------------------------------------------------
shift_pixels = 125      # horizontal shift to the RIGHT (in pixels)
shift_mode   = "zero"   # only "zero" is used for expanded canvas (no wrap here)

# ---------------------------------------------------------
# Utilities
# ---------------------------------------------------------
def render_image(arr, title="", cmap="jet", vmin=None, vmax=None, cbar_label="", phase=False, wrap=False):
    fig, ax = plt.subplots(figsize=(4.2, 4), dpi=120)
    if phase and not wrap and (vmin is None or vmax is None):
        vmin, vmax = -180, 180
    if wrap and (vmin is None or vmax is None):
        vmin, vmax = 0, 360
    im = ax.imshow(arr, cmap=cmap, origin="lower", vmin=vmin, vmax=vmax)
    cb = plt.colorbar(im, ax=ax)
    if cbar_label: cb.set_label(cbar_label)
    ax.set_title(title)
    ax.set_xlabel("x (px)")
    ax.set_ylabel("y (px)")
    # Ensure full array is shown
    ax.set_xlim(0, arr.shape[1])
    ax.set_ylim(0, arr.shape[0])
    fig.tight_layout()
    buf = BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    plt.close(fig)
    return buf.getvalue()

# ---------------------------------------------------------
# Build EXPANDED canvases and 32-field stack
# ---------------------------------------------------------
# Reconstruct complex base fields (16,H,W)
C16 = B_MAG * np.exp(1j * np.deg2rad(B_PHASE_ORIG))

# Expanded width to accommodate shift
W_new = W + shift_pixels

# Allocate expanded originals and expanded-shifted copies
C16_expanded       = np.zeros((16, H, W_new), dtype=np.complex128)
C16_shift_expanded = np.zeros((16, H, W_new), dtype=np.complex128)

# Place originals at [0:W] and shifted copies at [shift:shift+W]
C16_expanded[:, :, :W] = C16
C16_shift_expanded[:, :, shift_pixels:shift_pixels+W] = C16

# “Shifted+Added” version in the expanded canvas
C16_aug = C16_expanded + C16_shift_expanded  # (16,H,W_new)

# Final 32-field collection: first 16 expanded originals, next 16 expanded shifted+added
C32 = np.concatenate([C16_expanded, C16_aug], axis=0)  # (32,H,W_new)
MAG32 = np.abs(C32)
PHASE32 = np.angle(C32, deg=True)

# Update global dims
H, W = MAG32.shape[1], MAG32.shape[2]

# ---------------------------------------------------------
# Controls
# ---------------------------------------------------------
mode_dd = Dropdown(
    options=["Sum of Squares", "Sum"],
    value="Sum of Squares",
    description="Mode:",
    layout=Layout(width="220px")
)
scale_dd = Dropdown(
    options=["Linear", "Log"],
    value="Linear",
    description="Scale:",
    layout=Layout(width="220px")
)

# Default phases for 32 fields:
odd_values  = np.linspace(0, 360, 8, endpoint=False)   # [0,45,90,...,315]
even_values = (odd_values + 180) % 360                 # [180,225,270,...,135]
ph16 = np.empty(16)
ph16[0::2] = odd_values
ph16[1::2] = even_values
default_phases32 = np.concatenate([ph16, ph16])        # duplicate for fields 17–32

mag_sliders = [
    FloatSlider(value=1.0, min=0.0, max=3.0, step=0.01,
                description=f"Mag {i+1}", continuous_update=False,
                layout=Layout(width="300px"))
    for i in range(32)
]
phs_sliders = [
    IntSlider(value=int(default_phases32[i]), min=0, max=360, step=1,
              description=f"Phase {i+1}", continuous_update=False,
              layout=Layout(width="300px"))
    for i in range(32)
]
reset_btn = Button(description="Reset weights & phases", layout=Layout(width="250px"))

def reset_sliders(_=None):
    for s in mag_sliders: s.value = 1.0
    for i, s in enumerate(phs_sliders): s.value = int(default_phases32[i])
reset_btn.on_click(reset_sliders)

# ---------------------------------------------------------
# Combine fields (32) with weights & phase shifts
# ---------------------------------------------------------
def combine_fields(mode, mags, phs_deg):
    mags = np.asarray(mags, dtype=np.float64)                    # (32,)
    phs_rad = np.deg2rad(np.asarray(phs_deg, dtype=np.float64))  # (32,)

    phi0_rad = np.deg2rad(PHASE32)                # (32,H,W_new)
    mag_w   = mags[:, None, None] * MAG32         # (32,H,W_new)
    phase_t = phi0_rad + phs_rad[:, None, None]   # (32,H,W_new)
    complex_terms = mag_w * np.exp(1j * phase_t)  # (32,H,W_new)

    complex_sum = np.sum(complex_terms, axis=0)   # (H,W_new)
    phase_comb_deg = np.angle(complex_sum, deg=True)

    if mode == "Sum":
        mag_comb = np.abs(complex_sum)
    else:  # Sum of Squares
        mag_comb = np.sum(mag_w**2, axis=0)
    return mag_comb, phase_comb_deg

# ---------------------------------------------------------
# Outputs
# ---------------------------------------------------------
img_mag        = Image(format="png")
img_ph         = Image(format="png")
img_ph_wrapped = Image(format="png")

def update(_=None):
    mode = mode_dd.value
    scale_mode = scale_dd.value
    mags = [s.value for s in mag_sliders]
    phs  = [s.value for s in phs_sliders]

    mag_comb, phase_comb_deg = combine_fields(mode, mags, phs)
    phase_wrapped = np.mod(phase_comb_deg, 360.0)

    # Magnitude scaling
    if scale_mode == "Log":
        mag_plot = np.log10(np.clip(mag_comb, 1e-12, None))
        vmin, vmax = float(np.nanmin(mag_plot)), float(np.nanmax(mag_plot))
        title_mag = f"Combined Magnitude (Log10) — Mode: {mode}"
        cbar_lab  = "log10(|Magnitude|)"
    else:
        mag_plot = mag_comb
        vmin, vmax = float(np.nanmin(mag_plot)), float(np.nanmax(mag_plot))
        title_mag = f"Combined Magnitude — Mode: {mode}"
        cbar_lab  = "Magnitude (arb.)"

    img_mag.value = render_image(mag_plot, title=title_mag,
                                 cmap="jet", vmin=vmin, vmax=vmax, cbar_label=cbar_lab)
    img_ph.value  = render_image(phase_comb_deg, title="Combined Phase (original)",
                                 cmap="jet", phase=True, cbar_label="Phase (deg)")
    img_ph_wrapped.value = render_image(phase_wrapped, title="Combined Phase (wrapped 0–360°)",
                                        cmap="jet", wrap=True, cbar_label="Phase (deg)")

# Wire events
controls = [mode_dd, scale_dd, reset_btn] + mag_sliders + phs_sliders
for w in controls:
    if hasattr(w, "observe"):
        w.observe(update, names="value")

# Initial compute and layout
update()

left_mags  = VBox([Label("Magnitudes 1–16")] + mag_sliders[0:16])
right_mags = VBox([Label("Magnitudes 17–32")] + mag_sliders[16:32])
left_phs   = VBox([Label("Phases 1–16 (deg)")] + phs_sliders[0:16])
right_phs  = VBox([Label("Phases 17–32 (deg)")] + phs_sliders[16:32])

controls_top  = HBox([mode_dd, scale_dd, reset_btn], layout=Layout(gap="16px"))
controls_mags = HBox([left_mags, right_mags], layout=Layout(gap="24px"))
controls_phs  = HBox([left_phs, right_phs], layout=Layout(gap="24px"))
plots_box     = HBox([img_mag, img_ph, img_ph_wrapped])

display(VBox([
    Label(f"32-Field Combiner (Expanded width: {W} px = original {W - shift_pixels} + shift {shift_pixels})"),
    controls_top,
    controls_mags,
    controls_phs,
    plots_box
], layout=Layout(align_items="flex-start", gap="10px")))